###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es la Policía Nacional de Colombia

In [ ]:
import pandas as pd

###Archivo fpe5-yrmw.csv

Contiene información delitos Sexuales desde el 01 de enero del año 2010 al 30 de abril del año 2024.

In [ ]:
df = pd.read_csv("https://www.datos.gov.co/resource/fpe5-yrmw.csv?$limit=500000")
df.head()

<ipython-input-2-53592233e412>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("https://www.datos.gov.co/resource/fpe5-yrmw.csv?$limit=500000")


,departamento,municipio,codigo_dane,armas_medios,fecha_hecho,genero,grupo_etario,cantidad,delito
0,CAQUETÁ,SOLANO,18756000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,MENORES,1,ARTÍCULO 209. ACTOS SEXUALES CON MENOR DE 14 AÑOS
1,BOLÍVAR,CARTAGENA (CT),13001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADOLECENTES,1,ARTÍCULO 205. ACCESO CARNAL VIOLENTO
2,HUILA,LA PLATA,41396000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1,ARTÍCULO 205. ACCESO CARNAL VIOLENTO
3,SANTANDER,PIEDECUESTA,68547000,CONTUNDENTES,1/01/2010,FEMENINO,ADULTOS,1,ARTÍCULO 205. ACCESO CARNAL VIOLENTO
4,META,VILLAVICENCIO (CT),50001000,CONTUNDENTES,1/01/2010,FEMENINO,ADULTOS,1,ARTÍCULO 205. ACCESO CARNAL VIOLENTO


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319535 entries, 0 to 319534
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   departamento  319535 non-null  object
 1   municipio     319535 non-null  object
 2   codigo_dane   319535 non-null  object
 3   armas_medios  319535 non-null  object
 4   fecha_hecho   319535 non-null  object
 5   genero        319535 non-null  object
 6   grupo_etario  319535 non-null  object
 7   cantidad      319535 non-null  int64 
 8   delito        319535 non-null  object
dtypes: int64(1), object(8)
memory usage: 21.9+ MB


### Eliminación de columnas irrelevantes para el proyecto

In [ ]:
relevant_cols = ['codigo_dane', 'departamento', 'municipio', 'fecha_hecho', 'genero', 'grupo_etario', 'cantidad', 'delito']
sexual_crimes = df[relevant_cols]
sexual_crimes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319535 entries, 0 to 319534
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   codigo_dane   319535 non-null  object
 1   departamento  319535 non-null  object
 2   municipio     319535 non-null  object
 3   fecha_hecho   319535 non-null  object
 4   genero        319535 non-null  object
 5   grupo_etario  319535 non-null  object
 6   cantidad      319535 non-null  int64 
 7   delito        319535 non-null  object
dtypes: int64(1), object(7)
memory usage: 19.5+ MB


### Convertir columna fecha_hecho a tipo date

In [ ]:
# Hacer una copia explícita del DataFrame
sexual_crimes_ = sexual_crimes.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime usando el formato correcto
sexual_crimes_['fecha_hecho'] = pd.to_datetime(sexual_crimes_['fecha_hecho'], format='%d/%m/%Y', errors='coerce', dayfirst=True)

# Extraer solo el año
sexual_crimes_.loc[:, 'fecha_hecho'] = sexual_crimes_['fecha_hecho'].dt.year

### Verificar valores nulos

In [ ]:
sexual_crimes_.isnull().sum()

,0
codigo_dane,0
departamento,0
municipio,0
fecha_hecho,0
genero,0
grupo_etario,0
cantidad,0
delito,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [ ]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['genero', 'grupo_etario', 'delito', 'departamento', 'municipio']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(sexual_crimes_[column].unique())
    print()


Categorías en la columna 'genero':
['FEMENINO' 'MASCULINO' 'NO REPORTA' 'NO APLICA' '-' 'NO REPORTADO']

Categorías en la columna 'grupo_etario':
['MENORES' 'ADOLECENTES' 'ADULTOS' 'ADOLESCENTES' 'NO APLICA' 'NO REPORTA'
 '-' 'NO REPORTADO']

Categorías en la columna 'delito':
['ARTÍCULO 209. ACTOS SEXUALES CON MENOR DE 14 AÑOS'
 'ARTÍCULO 205. ACCESO CARNAL VIOLENTO'
 'ARTÍCULO 206. ACTO SEXUAL VIOLENTO'
 'ARTÍCULO 208. ACCESO CARNAL ABUSIVO CON MENOR DE 14 AÑOS'
 'ARTÍCULO 210 A. ACOSO SEXUAL'
 'ARTÍCULO 210. ACCESO CARNAL O ACTO SEXUAL ABUSIVO CON INCAPAZ DE RESISTIR'
 'ARTÍCULO 211. ACCESO CARNAL O ACTO SEXUAL ABUSIVO CON INCAPAZ DE RESISTIR (CIRCUNSTANCIAS AGRAVACIÓN'
 'ARTÍCULO 211. ACCESO CARNAL ABUSIVO CON MENOR DE 14 AÑOS (CIRCUNSTANCIAS AGRAVACIÓN)'
 'ARTÍCULO 211. ACTOS SEXUALES CON MENOR DE 14 AÑOS (CIRCUNSTANCIAS DE AGRAVACIÓN)'
 'ARTÍCULO 207. ACCESO CARNAL O ACTO SEXUAL EN PERSONA PUESTA EN INCAPACIDAD DE RESISTIR'
 'ARTÍCULO 211. ACCESO CARNAL VIOLENTO (CIRCUNSTANCIAS A

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [ ]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [ ]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    sexual_crimes_[col] = sexual_crimes_[col].apply(remove_accents_and_special_chars)

- Mejorar consistencia de las columnas 'genero' y 'grupo_etario'

In [ ]:
sexual_crimes_['genero'] = sexual_crimes_['genero'].replace({
    'NO REPORTA': 'NO REPORTADO',
    'NO APLICA': 'NO REPORTADO',
    '-': 'NO REPORTADO',
    'NO REPORTADO': 'NO REPORTADO'
})

In [ ]:
sexual_crimes_['grupo_etario'] = sexual_crimes_['grupo_etario'].replace({
    'ADOLECENTES': 'ADOLESCENTES',
    'NO REPORTA': 'NO REPORTADO',
    'NO APLICA': 'NO REPORTADO',
    '-': 'NO REPORTADO',
    'NO REPORTADO': 'NO REPORTADO'
})

- Codificación de algunas variables categóricas

### Ajustar columna 'codigo_dane' para que coincida con el campo 'dept_mpio_code' de la tabla municipalities de la base de datos, que guarda toda la informacion de georeferenciacion de los municipios

 -  Cargar los datos con códigos reales de los municipios

Como producto de una consulta a la base de datos del proyecto que se esta construyendo (Tablas departments y municipalities) se creo el archivo csv que se carga en la siguiente celda, y que incluye los nombres de los departamentos y municipios con sus respectivos codigos, generados por el DANE (Estos codigos son los reales)

In [ ]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "codigo_dane" en el df sexual_crimes_

In [ ]:
# Asegurarnos de que todos los valores en 'codigo_dane' sean strings
sexual_crimes_['codigo_dane'] = sexual_crimes_['codigo_dane'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = sexual_crimes_['codigo_dane'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

False

In [ ]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [ 8  7  9 10]


In [ ]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,no_digitos_codigo_dane,count,percentage
0,8,272216,0.851913
1,7,46143,0.144407
2,10,1175,0.003677
3,9,1,0.000003


In [ ]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = sexual_crimes_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['codigo_dane']])
    print()

Muestra de registros con longitud 8:
  codigo_dane
0    18756000
1    13001000
2    41396000
3    68547000
4    50001000

Muestra de registros con longitud 7:
   codigo_dane
62     8001000
65     5490000
76     5837000
93     8758000
99     8758000

Muestra de registros con longitud 10:
       codigo_dane
278619  01/12/2022
278620  01/12/2022
278621  01/12/2022
278622  01/12/2022
278623  01/12/2022

Muestra de registros con longitud 9:
       codigo_dane
117400   NO APLICA



### Nota:

De lo anterior se notan claras inconsistencias en la columna 'codigo_dane' del df, el 85% tiene 8 dígitos (al parecer se le adicionaron 3 ceros al final del código que realmente es de 5 dígitos), el 14,4% tiene solamente 7 dígitos (al parecer el cero a la izquierda de los códigos se suprimió), en el 0.36% de los códigos en vez del código aparece una fecha digitada y en 1 aparece la palabra NO APLICA

  - Verificar si los codigo_dane de 7 dígitos corresponden a departamentos que se identifican con 1 dígito para validar la teoria de que al generar el dataset se les suprimió el cero a la izquierda

In [ ]:
# Filtrar las filas donde 'codigo_dane' tiene 7 dígitos
filtrado = sexual_crimes_[sexual_crimes_['codigo_dane'].str.len() == 7]

# Obtener las categorías únicas de la columna 'departamento'
categorias_departamento = filtrado['departamento'].unique()

# Imprimir las categorías
print(categorias_departamento)

['ATLANTICO' 'ANTIOQUIA']


Efectivamente ANTIOQUIA Y ATLANTICO son los unicos departamentos que tienen codigo Dane de un dígito, 5 y 8 respectivamente.

  - Adicionar un cero a los codigo_dane de 7 dígitos

In [ ]:
# Función que agrega un '0' a la izquierda si la longitud del string es 7
def add_zero_if_length_7(codigo):
    if len(codigo) == 7:
        return '0' + codigo
    return codigo

# Aplicar la función a la columna 'codigo'
sexual_crimes_['codigo_dane'] = sexual_crimes_['codigo_dane'].apply(add_zero_if_length_7)

In [ ]:
# Verificar cuantos codigo_dane de 7 dígitos quedaron
len(sexual_crimes_[sexual_crimes_['codigo_dane'].str.len() == 7])

0

  - Quitar los ultimos 3 ceros de los codigo_dane que tienen 8 dígitos

In [ ]:
# Función que corta los últimos 3 caracteres si la longitud del string es 8
def trim_last_3_if_length_8(codigo):
    if len(codigo) == 8:
        return codigo[:5]  # Dejar solo los primeros 5 caracteres
    return codigo

# Aplicar la función a la columna 'codigo'
sexual_crimes_['codigo_dane'] = sexual_crimes_['codigo_dane'].apply(trim_last_3_if_length_8)

In [ ]:
# Verificar cuantos codigo_dane de 8 dígitos quedaron
len(sexual_crimes_[sexual_crimes_['codigo_dane'].str.len() == 8])

0

### Solucionar los codigo_dane que en el dataset fueron digitados erroneamente como fechas y NO APLICA

  a) Iniciamos ajustando la columna de nombre de departamento de los dataframe: sexual_crimes_ y dept_mpios_codes para que coincidan en formato

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños en el df: dept_mpios_codes

In [ ]:
# Aplicar funcion a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

In [ ]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [ ]:
# Comparar listas de departamento
compare_lists(sexual_crimes_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en sexual_crimes_", "Departamentos dept_mpios_codes")

Departamentos en sexual_crimes_ que no están en Departamentos dept_mpios_codes:
{'VALLE', 'GUAJIRA', 'SAN ANDRES'}


  - Solucionar discrepancias en nombres de departamento

In [ ]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = {'SAN ANDRES': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
                 'VALLE': 'VALLE DEL CAUCA',
                 'GUAJIRA':'LA GUAJIRA'}

# Reemplazar los nombres incorrectos  el dataframe sexual_crimes_
sexual_crimes_['departamento'] = sexual_crimes_['departamento'].replace(depto_mapping)

 - Confirmar que discrepancia se solucionó

In [ ]:
# Comparar listas de departamento
compare_lists(sexual_crimes_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en sexual_crimes_", "Departamentos dept_mpios_codes")

Departamentos en sexual_crimes_ que no están en Departamentos dept_mpios_codes:
set()


  b) Aplicar coincidencia difusa para combinacion departamento + municipio en los dos dataframe

Realizar coincidencia difusa (fuzzy matching), significa que puede comparar dos cadenas de texto (strings) y medir su similitud, incluso si no son exactamente iguales. Esta técnica es muy útil en este caso porque si bien los nombres de los departamentos estan debidamente ajustados en los dos df,  los nombres de los municipios pueden tener diferencias (errores de tipeo, variantes en nombres, etc.).

El utilizar la columna del nombre del departamento, es importante en este caso porque en colombia existen municipios con el mismo nombre.


 - Instalar la thefuzz en el entorno de google colab, para realizar coincidencia difusa

In [ ]:
!pip install thefuzz

- Aplicar thefuzz para coincidencia difusa

In [ ]:
# Crear una clave única de departamento + municipio en ambos datasets
sexual_crimes_['dept_mpio'] = sexual_crimes_['departamento'] + '_' + sexual_crimes_['municipio']
dept_mpios_codes['dept_mpio'] = dept_mpios_codes['dept_name'] + '_' + dept_mpios_codes['mupio_name']

In [ ]:
# Crear un diccionario de municipios y códigos basado en dept_mpios_codes
municipios_dict = dict(zip(dept_mpios_codes['dept_mpio'], dept_mpios_codes['dept_mpio_code']))

In [ ]:
from thefuzz import process

# Funcion para Usar fuzzy matching (thefuzz)
def get_best_match(row, municipios_dict, threshold=80):
    dept_mpio_sexual = row['departamento'] + '_' + row['municipio']

    # Buscar la mejor coincidencia en dept_mpios_codes usando fuzzy matching
    best_match, score = process.extractOne(dept_mpio_sexual, municipios_dict.keys())

    # Si la similitud supera el umbral definido, devolver el código mapeado
    if score >= threshold:
        return municipios_dict[best_match]
    else:
        return row['codigo_dane']  # Mantener el código original si no hay coincidencia segura


In [ ]:
# Aplicar la función solo a las filas donde el 'codigo_dane' no tenga 5 dígitos
sexual_crimes_['codigo_dane_corr'] = sexual_crimes_.apply(
    lambda row: get_best_match(row, municipios_dict, threshold=80) if len(row['codigo_dane']) != 5 else row['codigo_dane'],
    axis=1
)

  - Verificar resultados obtenidos

In [ ]:
sexual_crimes_['codigo_dane_corr'].apply(len).nunique()

2

In [ ]:
sexual_crimes_['codigo_dane_corr'].apply(len).value_counts()

,count
codigo_dane_corr,
5,319534
9,1


Sigue habiendo una clave incorrecta de una longitud de 9 dígitos o caracteres. como se aprecia es la fila 117400 del dataset original, corresponde a un caso del departamento de GUAINIA pero no tiene el municipio.

In [ ]:
sexual_crimes_[sexual_crimes_['codigo_dane_corr'].apply(len) == 9]

,codigo_dane,departamento,municipio,fecha_hecho,genero,grupo_etario,cantidad,delito,dept_mpio,codigo_dane_corr
117400,NO APLICA,GUAINIA,NO APLICA,2017,FEMENINO,MENORES,1,ARTICULO 218. PORNOGRAFIA CON MENORES,GUAINIA_NO APLICA,NO APLICA


Procedemos a eliminar el anterior registro pues no existen pistas para saber de que municipio se trata


In [ ]:
# Eliminar las filas con índice 117400
sexual_crimes_ = sexual_crimes_.drop(index=[117400])

- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [ ]:
# Comparar listas de códigos
compare_lists(sexual_crimes_['codigo_dane_corr'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en sexual_crimes_", "Códigos de municipios dept_mpios_codes")

Departamentos en sexual_crimes_ que no están en Departamentos dept_mpios_codes:
{'52000'}


  - Identifcar los registros bajo este código que no corresponde a ningun código real de municipio

In [ ]:
sexual_crimes_[sexual_crimes_['codigo_dane_corr'] == '52000']

,codigo_dane,departamento,municipio,fecha_hecho,genero,grupo_etario,cantidad,delito,dept_mpio,codigo_dane_corr
288337,52000,NARINO,-,2023,MASCULINO,ADULTOS,1,ARTICULO 210 A. ACOSO SEXUAL,NARINO_-,52000
288338,52000,NARINO,-,2023,FEMENINO,ADOLESCENTES,1,ARTICULO 206. ACTO SEXUAL VIOLENTO,NARINO_-,52000
306510,52000,NARINO,NO REPORTADO,2023,FEMENINO,MENORES,1,ARTICULO 209. ACTOS SEXUALES CON MENOR DE 14 ANOS,NARINO_NO REPORTADO,52000
306511,52000,NARINO,NO REPORTADO,2023,FEMENINO,ADULTOS,1,ARTICULO 210 A. ACOSO SEXUAL,NARINO_NO REPORTADO,52000
317989,52000,NARINO,-,2024,MASCULINO,ADULTOS,1,ARTICULO 208. ACCESO CARNAL ABUSIVO CON MENOR ...,NARINO_-,52000


Son 5 municipios del departamento de Narino que no estan identificados, procedemos a eliminar los registros del dataset

In [ ]:
# Eliminar filas donde 'codigo_dane_corr' es igual a '52000'
sexual_crimes_ = sexual_crimes_[sexual_crimes_['codigo_dane_corr'] != '52000']

### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [ ]:
# Eliminar columnas innecesarias
columns_to_drop = ['codigo_dane', 'departamento', 'municipio', 'dept_mpio']
final_sexual_crimes = sexual_crimes_.drop(columns=columns_to_drop)

In [ ]:
# Adicionar columna para trazabilidad de la fuente
final_sexual_crimes['source_id'] = 6

In [ ]:
final_sexual_crimes.columns

Index(['fecha_hecho', 'genero', 'grupo_etario', 'cantidad', 'delito',
       'codigo_dane_corr', 'source_id'],
      dtype='object')

In [ ]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'fecha_hecho': 'year_of_incident',
    'genero': 'sex',
    'grupo_etario': 'age_group',
    'cantidad': 'amount',
    'delito': 'crime_type',
    'codigo_dane_corr': 'dane_code',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_sexual_crimes.rename(columns=translation_map, inplace=True)

In [ ]:
#Estructura final del dataset a integrar a la base de datos
final_sexual_crimes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 319529 entries, 0 to 319534
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   year_of_incident  319529 non-null  int32 
 1   sex               319529 non-null  object
 2   age_group         319529 non-null  object
 3   amount            319529 non-null  int64 
 4   crime_type        319529 non-null  object
 5   dane_code         319529 non-null  object
 6   source_id         319529 non-null  int64 
dtypes: int32(1), int64(2), object(4)
memory usage: 18.3+ MB


## Salvar en archivo csv en el drive

In [ ]:
# Guardar en archivos CSV en el drive
final_sexual_crimes.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/sexual_crimes.csv', index=False)